In [3]:
import datetime
import pandas as pd
import numpy as np
import math, sys, os
import scipy.stats

filename = 'data/dataset-labeled-2.csv'
dataset = pd.read_csv(filename)
print('data loaded')

data loaded


In [ ]:
dataset[dataset.prev_0>0]['prev_0'].quantile(0.25)


In [4]:
print(dataset['amp_60'].quantile(0.25))
print(dataset['amp_60'].quantile(0.5))
print(dataset['amp_60'].quantile(0.75))

22.65
32.88
49.67


In [ ]:
q="(trend_60==0) & (trend_30==0) & (trend_20==0) & (trend_10==0) & (trend_5==0) & (trend_3==0) & (amp_60<22.65) & (amp_10<5.98) & (prev_3>0) & (prev_2<=0) & (prev_1<0 & prev_1>=-1.59) & (prev_0>1.52)"
dataset[dataset.eval(q)].shape